In [1]:

from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


**importing tools_conversion1.py file**

In [14]:
from google.colab import files 
src = list(files.upload().values())[0] 

Saving tools_conversion1.py to tools_conversion1.py


In [ ]:
!pip install soundfile

In [15]:
#deleting text files from datasets
import glob
import os
files=glob.glob("/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/clean_voice/*.txt")

for file in files:
  os.remove(file)

  

In [ ]:
#copying files
import os, shutil

root_src_dir = "/content/drive/MyDrive/Colab Notebooks/open_unmix/ESC-50/audio"
root_dst_dir = "/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/noise"

for src_dir, dirs, files in os.walk(root_src_dir):
    dst_dir = src_dir.replace(root_src_dir, root_dst_dir, 1)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst_dir, file_)
        if os.path.exists(dst_file):
            os.remove(dst_file)
        shutil.copy(src_file, dst_dir)

**Mix clean speech with noised voice** and prepare the input data (spectrograms,time series..) before the training process

In [1]:
import os
import librosa
import soundfile as sf
import numpy as np
from tools_conversion1 import *

""" this function aims at preparing datasets for training. """

def prepare_noised_voice(noise_dir, voice_dir, sample_rate,min_duration, 
                         frame_length, hop_length_frame, hop_length_frame_noise,
                         nb_samples, n_fft, hop_length_fft,files_path):

    noise_files = os.listdir(noise_dir)
    voice_files = os.listdir(voice_dir)

    nb_voice_files = len(voice_files)
    nb_noise_files = len(noise_files)
   
    # convert noise and voice to numpy
    noise = audio_to_numpy(noise_dir, noise_files, sample_rate,
                                     frame_length, hop_length_frame_noise, min_duration)

    voice = audio_to_numpy(voice_dir,voice_files,
                                     sample_rate, frame_length, hop_length_frame, min_duration)

    # mix some clean voices with random noises 
    res_voice, res_noise, res_noisy_voice = mix_voice_noise(
            voice, noise, nb_samples, frame_length)
    
    # save the audio generated
    noisy_voice_long = res_noisy_voice.reshape(1, nb_samples * frame_length)
    sf.write(files_path[1] + 'noisy_voice.wav', noisy_voice_long[0, :], sample_rate)
    voice_long = res_voice.reshape(1, nb_samples * frame_length)
    sf.write(files_path[1] + 'voice.wav', voice_long[0, :], sample_rate)
    noise_long = res_noise.reshape(1, nb_samples * frame_length)
    sf.write(files_path[1] + 'noise.wav', noise_long[0, :], sample_rate)

    # Squared spectrogram dimensions
    dim_square_spec = int(n_fft / 2) + 1

    # Create Amplitude and phase spectrograms of the voice generated
    mat_amp_db_voice,  mat_pha_voice = numpy_to_matSpectrogram(
            res_voice, dim_square_spec, n_fft, hop_length_fft)
    mat_amp_db_noise,  mat_pha_noise = numpy_to_matSpectrogram(
            res_noise, dim_square_spec, n_fft, hop_length_fft)
    mat_amp_db_noisy_voice,  mat_pha_noisy_voice = numpy_to_matSpectrogram(
            res_noisy_voice, dim_square_spec, n_fft, hop_length_fft)

    # Save results in the drive

    #Time series
    np.save(files_path[0] + 'noisy_voice_timeserie', res_noisy_voice)
    np.save(files_path[0] + 'voice_timeserie', res_voice)
    np.save(files_path[0] + 'noise_timeserie', res_noise)

    #Magnitude spectrograms
    np.save(files_path[2] + 'noisy_voice_amp', mat_amp_db_noisy_voice)
    np.save(files_path[2] + 'voice_amp', mat_amp_db_voice)
    np.save(files_path[2] + 'noise_amp', mat_amp_db_noise)
    
    #Phase spectrograms
    np.save(files_path[2] + 'noisy_voice_pha', mat_pha_noisy_voice)
    np.save(files_path[2] + 'voice_pha', mat_pha_voice)
    np.save(files_path[2] + 'noise_pha', mat_pha_noise)


**Creating input data**

In [2]:
#data_mode

import os

        #folder containing noises
noise_dir = '/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/noise/'
        #folder containing clean voices
voice_dir = '/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/clean_voice/'
        #path to save time series
time_serie_path = '/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/time_series/'
        #path to save sounds
sound_path = '/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/sound/'
        #path to save spectrograms
spectrogram_path = '/content/drive/MyDrive/Colab Notebooks/speech_enhancement/data/train/spectrogram/'
files_path=[time_serie_path,sound_path,spectrogram_path]

sample_rate = 8000
        # Minimum duration of each window
min_duration = 1
        
frame_length = 8064
        # hop length for clean voice files
hop_length_frame = 8064
        # hop length for noise files
hop_length_frame_noise = 5000
        # total number of frames to train
nb_samples = 5800
        #nb of points for fft
n_fft = 255
        #hop length for fft
hop_length_fft = 63

prepare_noised_voice(noise_dir, voice_dir,sample_rate,min_duration,
                     frame_length, hop_length_frame, hop_length_frame_noise, 
                     nb_samples, n_fft, hop_length_fft,files_path)


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
